In [12]:
import pandas as pd 
import random
from tqdm import tqdm

N_COLS = 640
N_ROWS = 480

# n positives and n negatives for each scale
PATCHES_EACH_ROW = 32
SCALES = [32,64,128,256]

DATA = "../data/naodevils/raw/spqr_autolabel_and_manual_train.csv"

df = pd.read_csv(DATA)


In [13]:
import pandas as pd
import random
from tqdm import tqdm

def is_ball_within_patch(row, patch_coordinates, patch_dim):
    return row["frame_contains_ball"] and \
        patch_coordinates[0] < row["center_x"] < patch_coordinates[0] + patch_dim and \
        patch_coordinates[1] < row["center_y"] < patch_coordinates[1] + patch_dim 

patches_list = []

for _, sample in tqdm(df.iterrows(), total=len(df)):

    for scale in SCALES:

        for _ in range(PATCHES_EACH_ROW):

            # GENERATE NEGATIVE
            while True:
                r_x = random.randint(0, N_COLS - scale - 1)
                r_y = random.randint(0, N_ROWS - scale - 1)

                if not is_ball_within_patch(sample, (r_x, r_y), scale):
                    row = sample.to_dict()
                    row["patch_x"] = r_x
                    row["patch_y"] = r_y
                    row["patch_size"] = scale
                    row["patch_contains_ball"] = False
                    patches_list.append(row)
                    break

            # GENERATE POSITIVE
            while sample["frame_contains_ball"]:
                min_x = max(int(sample["center_x"]) - scale + 1, 0)
                min_y = max(int(sample["center_y"]) - scale + 1, 0)
                
                scaled_radius =  32 * sample["radius"] / scale

                if int(sample["center_x"]) <= 1 or int(sample["center_y"]) <= 1 or scaled_radius < 4 or scaled_radius > 16:
                    break

                r_x = random.randint(min_x, int(sample["center_x"]) - 1)
                r_y = random.randint(min_y, int(sample["center_y"]) - 1)

                if is_ball_within_patch(sample, (r_x, r_y), scale):
                    row = sample.to_dict()
                    row["patch_x"] = r_x
                    row["patch_y"] = r_y
                    row["patch_size"] = scale
                    row["patch_contains_ball"] = True
                    patches_list.append(row)
                    break

patches_df = pd.DataFrame(patches_list)
patches_df.to_csv(DATA[:-4]+"_patchified.csv", index=False)


100%|██████████| 8620/8620 [00:22<00:00, 377.75it/s]


In [15]:
patches_df = pd.read_csv(DATA[:-4]+"_patchified.csv")
patches_df

,id,image,frame_contains_ball,center_x,center_y,radius,patch_x,patch_y,patch_size,patch_contains_ball
0,0,08586_RoboCup2019_HULKs_Daredevil-1st_16610745...,True,440.007727,400.850526,51.08463,384,36,32,False
1,0,08586_RoboCup2019_HULKs_Daredevil-1st_16610745...,True,440.007727,400.850526,51.08463,154,341,32,False
2,0,08586_RoboCup2019_HULKs_Daredevil-1st_16610745...,True,440.007727,400.850526,51.08463,271,127,32,False
3,0,08586_RoboCup2019_HULKs_Daredevil-1st_16610745...,True,440.007727,400.850526,51.08463,52,45,32,False
4,0,08586_RoboCup2019_HULKs_Daredevil-1st_16610745...,True,440.007727,400.850526,51.08463,111,171,32,False
...,...,...,...,...,...,...,...,...,...,...
1348891,8619,00983_RoboCup2019_HTWK-Leipzig_Venom-2nd_22397...,False,NaN,NaN,NaN,65,144,256,False
1348892,8619,00983_RoboCup2019_HTWK-Leipzig_Venom-2nd_22397...,False,NaN,NaN,NaN,74,89,256,False
1348893,8619,00983_RoboCup2019_HTWK-Leipzig_Venom-2nd_22397...,False,NaN,NaN,NaN,125,15,256,False
1348894,8619,00983_RoboCup2019_HTWK-Leipzig_Venom-2nd_22397...,False,NaN,NaN,NaN,284,215,256,False
